In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp

## Overall Shape

In [ ]:
def lennard_jones(x, epsilon, sigma, bound, cutoff):
    s = 4*epsilon*(12*(sigma/x)**11 - 6*(sigma/x)**5)
    q = x<=cutoff
    return q * np.minimum(s, [bound])

def stepwise(x, attracting, repelling, sigma, interaction_range, cutoff):
    return (repelling*(x<=sigma) - attracting*(sigma<x))*(x<=cutoff)

sigma = 1.0
epsilon = 1.0
bound = 1.515
cutoff = 2.5

import scipy as sp
xdata = [0, sigma, 2*sigma, cutoff]
ydata = [bound, 0, -epsilon, 0]
fit_func = lambda x, a0, a2, a3, a4: a0 + a2*x**2 + a3*x**3 + a4*x**4
fit_params, _ = sp.optimize.curve_fit(fit_func, xdata, ydata)

fig, ax = plt.subplots()
x = np.linspace(0.001, 3.5*sigma, 200)
y1 = lennard_jones(x, epsilon, sigma, bound, cutoff)
y3 = stepwise(x, epsilon, epsilon, sigma, bound, cutoff)
y2 = fit_func(x, *fit_params)

ax.plot(x, y1, label="Lennard-Jones")
ax.plot(x, y2, label="Fit")
ax.plot(x, y3, label="Linear Jones")

plt.show(fig)

## Neighbors

In [ ]:
def func(x):
    n = 6
    alpha = 4
    s = (x/alpha)**n/(1+(x/alpha)**n)
    return s

x1 = np.linspace(0, 8)
x2 = np.arange(0, 8)
y1 = func(x1)
y2 = func(x2)

fig, ax = plt.subplots()
ax.plot(x1, y1)
ax.scatter(x2, y2)
plt.show(fig)

In [ ]:
def calculate_potential_1(r, potential_strength, repelling_strength, cell_radius, ext_radius, interaction_range):
    sigma = r / (cell_radius + ext_radius)
    bound = 1.0 / (1.0 + sigma**3)
    spatial_cutoff = np.heaviside(1.0 + (interaction_range + cell_radius + ext_radius - r), 0)

    # Calculate the strength of the interaction with correct bounds
    strength_relative = np.clip((1.0 / sigma)**4 - 2*(1.0 / sigma)**2, -1, bound)
    attractive_force = np.clip(strength_relative, -np.infty, 0.0)
    repelling_force = np.clip(strength_relative, 0.0, np.infty)
    return spatial_cutoff * (potential_strength*attractive_force + repelling_strength * repelling_force)

def calculate_potential_2(r, potential_strength, cell_radius, ext_radius, interaction_range):
    sigma = r / (cell_radius + ext_radius)
    bound = 4.0 + 1.0 / sigma
    spatial_cutoff = np.sign(1.0 + (interaction_range + cell_radius + ext_radius - r)) * 0.5

    # Calculate the strength of the interaction with correct bounds
    strength_relative = np.clip((1.0 / sigma)**2 - (1.0 / sigma)**4, -bound, bound)
    # attractive_force = np.clip(strength_relative, -np.infty, 0.0)
    # repelling_force = np.clip(strength_relative, 0.0, np.infty)
    return - potential_strength * strength_relative * spatial_cutoff

cell_radius = 1.0
interaction_range = 0.8
ext_radius = 1.0
potential_strength = 0.003
repelling_strength = 0.035

x = np.linspace(0.1, 2*(cell_radius + interaction_range + ext_radius), 200)
y1 = calculate_potential_1(x, potential_strength, repelling_strength, cell_radius, ext_radius, interaction_range)
y2 = calculate_potential_2(x, 0.01, cell_radius, ext_radius, interaction_range)

fig, ax = plt.subplots()
ax.plot(x, y1, label="Relative Potential Strength 1", color="black")
ax.plot(x, y2, label="Relative Potential Strength 2", color="red")
ax.legend()
plt.show(fig)